In [1]:
from google.colab import files

In [2]:
uploaded = files.upload()

Saving model.h5 to model.h5


In [3]:
!pip install scipy==1.1.0

     |████████████████████████████████| 31.2 MB 1.6 MB/s 
  Attempting uninstall: scipy
    Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-1.4.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pymc3 3.11.4 requires scipy>=1.2.0, but you have scipy 1.1.0 which is incompatible.
plotnine 0.6.0 requires scipy>=1.2.0, but you have scipy 1.1.0 which is incompatible.
jax 0.2.25 requires scipy>=1.2.1, but you have scipy 1.1.0 which is incompatible.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [4]:
import numpy as np
import cv2
from scipy.misc import imresize
from moviepy.editor import VideoFileClip
from tensorflow import keras

In [5]:
model = keras.models.load_model('model.h5')

In [27]:
class lanes():
  def __init__(self):
    self.recent_fit = []
    self.avg_fit = []


def road_lines(image):


  small_img = imresize(image, (80, 160, 3))# Reszing the image with 3 Channels which means our image is coloured
  small_img = np.array(small_img)
  small_img = small_img[None,:,:,:]#Adding another Dimension to the image


  prediction = model.predict(small_img)[0] * 255

  lanes.recent_fit.append(prediction)

  if len(lanes.recent_fit) > 5:
    lanes.recent_fit = lanes.recent_fit[1:]


  lanes.avg_fit = np.mean(np.array([i for i in lanes.recent_fit]), axis = 0)

  blanks = np.zeros_like(lanes.avg_fit).astype(np.uint8)

  lane_drawn = np.dstack((blanks, lanes.avg_fit, blanks))

  lane_image = imresize(lane_drawn, (720, 1280, 3))
  result = cv2.addWeighted(image, 1, lane_image, 1, 0)

  return result

In [13]:
uploaded = files.upload()

Saving lanes_clip.mp4 to lanes_clip.mp4


In [28]:
video_input = VideoFileClip('lanes_clip.mp4')
video_output = 'lanes_output_clip.mp4'

lanes = Lanes()

video_clip = video_input.fl_image(road_lines)
video_clip.write_videofile(video_output)

[MoviePy] >>>> Building video lanes_output_clip.mp4
[MoviePy] Writing audio in lanes_output_clipTEMP_MPY_wvf_snd.mp3


100%|██████████| 572/572 [00:00<00:00, 1448.76it/s]

[MoviePy] Done.
[MoviePy] Writing video lanes_output_clip.mp4



100%|█████████▉| 1297/1298 [02:42<00:00,  8.00it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: lanes_output_clip.mp4 

